<a href="https://colab.research.google.com/github/Yoon-Joon-soo/Kobert_for_ai/blob/master/Deit_Hugging_Face_15%2C16%2C17%2C18_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [4]:
!pip install transformers torch

In [5]:
!pip install gdown

In [6]:
!pip install --upgrade --no-cache-dir gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [7]:
import gdown
import zipfile

# Google 드라이브 공유 링크에서 파일 ID 추출 (공유 가능한 링크를 사용하면 "uc?id=" 이후의 부분이 파일 ID입니다)
file_id = '1yPrs3PbyvKJ--H7vgl5S5x50Jx1mcCX0'

# 다운로드할 파일의 링크 생성
file_url = f'https://drive.google.com/uc?id={file_id}'

# 파일 다운로드
output_path = 'FERPlus_Dataset.zip'
gdown.download(file_url, output_path, quiet=False)

# 압축 해제
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall('FERPlus_Dataset')

Downloading...
From (uriginal): https://drive.google.com/uc?id=1yPrs3PbyvKJ--H7vgl5S5x50Jx1mcCX0
From (redirected): https://drive.google.com/uc?id=1yPrs3PbyvKJ--H7vgl5S5x50Jx1mcCX0&confirm=t&uuid=b632f09b-d330-4c98-91ec-d09e8a230e6c
To: /content/FERPlus_Dataset.zip
100%|██████████| 62.9M/62.9M [00:00<00:00, 72.2MB/s]


In [8]:
# Data_loader.py

import os
import torch
from torchvision import datasets, transforms
from PIL import Image
import numpy as np
import cv2

class FERPlusDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.train = train

        if self.train:
            self.data_dir = os.path.join(self.root_dir, 'train')
        else:
            self.data_dir = os.path.join(self.root_dir, 'test')

        self.classes = sorted(os.listdir(self.data_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        self.samples = []
        for cls_name in self.classes:
            cls_dir = os.path.join(self.data_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                img_path = os.path.join(cls_dir, img_name)
                self.samples.append((img_path, self.class_to_idx[cls_name]))


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, target = self.samples[idx]
        img = Image.open(img_path).convert('RGB')  # 허깅 페이스에서 제공하는 deit 라이브러리가 컬러 이미지만을 취급하기 때문에 컬러로 변경

        if self.transform:
            img = self.transform(img)

        return img, target



In [9]:
# Data_preprosessing.py

import numpy as np
import cv2
import os
import glob

# cv2 모듈로 이미지 읽는 함수 정의
class compute_mean_and_std():
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_paths = glob.glob(os.path.join(root_dir, '**/*.*'), recursive=True)

    def read_img(self, file_path):
        img_arr = cv2.imread(file_path, cv2.IMREAD_COLOR)
        return img_arr

    def get_mean_and_std(self):
        # global mean 구하기
        global_mean = 0
        global_var = 0

        for img in self.image_paths:
            img_result = self.read_img(img) / 255
            global_mean += img_result.reshape(-1, 3).mean(axis=0)

        global_mean /= len(self.image_paths)

        # global std 구하기
        for img in self.image_paths:
            img_result = self.read_img(img) / 255
            global_var += ((img_result.reshape(-1, 3) - global_mean) ** 2).mean(axis=0)

        global_var /= len(self.image_paths)
        global_std = np.sqrt(global_var)

        print(global_mean, global_std)

        return global_mean, global_std

In [10]:
#fine_tuning.py

from torch import torch


def fine_tuning(model, num_classes, num_epochs, train_data_loader):
    model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=num_classes)

    learning_rate = 0.001


    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    i=1

    for epoch in range(num_epochs):
        for inputs, labels in train_data_loader:
            optimizer.zero_grad()
            outputs = model(inputs).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            i = i+1
            if(i%225==0):
              print(f'This is {i/225}th iteration for fine-tuning.\n')




    # 검증 데이터에 모델 적용 및 성능 평가
    # (정확도, F1 점수 등)
    #model.eval()
    #with torch.no_grad():
        #torch.save(model.state_dict(), 'fine_tuned_model.pth')


In [11]:
# model_loader.py

from transformers import DeiTForImageClassification

def load_model(model_name):
    model = DeiTForImageClassification.from_pretrained(model_name)


    return model

In [12]:
#test_model.py

from torch import torch
from sklearn.metrics import accuracy_score

def test_model(model, test_data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_data_loader:
            outputs = model(inputs).logits  # ImageClassifierOutput에서 logits 텐서 추출
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())

    print('Test Finish!!')
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy


In [13]:
# main.py
from torch import torch


root_dir = '/content/FERPlus_Dataset'
batch_size = 128
model_name = "facebook/deit-base-distilled-patch16-224"

# 이미지 크기 조정 변환
desired_size = 224
resize_transform= transforms.Resize((desired_size,desired_size))

desired_padding = 88
# 패딩을 추가하는 과정
padding_transform = transforms.Compose([
    #transforms.ToPILImage(), # Tensor를 PIL 이미지로 변환
    transforms.Pad(padding=desired_padding, fill=0, padding_mode= 'constant'), # desired_padding에 맞게 패딩 추가
])

# 데이터 정규화 즉, 평균과 표준편차 계산해서 반환하기
mean_std_calculator = compute_mean_and_std(root_dir)
mean, std = mean_std_calculator.get_mean_and_std()

# 데이터 전처리 설정
data_transform = transforms.Compose([
    padding_transform, # padding
    resize_transform,  # resizing
    transforms.Grayscale(num_output_channels=3),  # 이미지를 흑백에서 RGB로 변환
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# FERPlus 데이터 셋 로드
train_dataset = FERPlusDataset(root_dir, train=True, transform=data_transform)
test_dataset = FERPlusDataset(root_dir, train=False, transform=data_transform)

# 데이터 로더 설정 (배치 크기, 셔플 등을 설정하세요)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

print(f"class number: {len(train_dataset.classes)}")
print(f"data number: {len(train_dataset)}")
print(f"batch number: {len(train_loader)}")
print(f"test data number : {len(test_dataset)}")
print(f"test batch number : {len(test_loader)}")


# 모델 로드
model = load_model(model_name)

# 저장된 모델 가중치 불러오기
model_state_dict = torch.load('/content/drive/MyDrive/model_weights_after_new_14epochs.pth')

# 모델의 classifier 레이어를 변경하여 클래스 수 조정
num_classes = len(train_dataset.classes)
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=num_classes)

# 추가 에폭 학습
num_epochs_additional = 1
fine_tuning(model, num_classes, num_epochs_additional, train_loader)

# 저장된 모델 가중치 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/model_weights_after_new_15epochs.pth')

# 테스트
accuracy = test_model(model, test_loader)
print(f"Test Accuracy after 15 epochs: {accuracy}")

# 추가 에폭 학습
num_epochs_additional = 1
fine_tuning(model, num_classes, num_epochs_additional, train_loader)

# 저장된 모델 가중치 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/model_weights_after_new_16epochs.pth')

# 테스트
accuracy = test_model(model, test_loader)
print(f"Test Accuracy after 16 epochs: {accuracy}")

# 추가 에폭 학습
num_epochs_additional = 1
fine_tuning(model, num_classes, num_epochs_additional, train_loader)

# 저장된 모델 가중치 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/model_weights_after_new_17epochs.pth')

# 테스트
accuracy = test_model(model, test_loader)
print(f"Test Accuracy after 17 epochs: {accuracy}")

# 추가 에폭 학습
num_epochs_additional = 1
fine_tuning(model, num_classes, num_epochs_additional, train_loader)

# 저장된 모델 가중치 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/model_weights_after_new_18epochs.pth')

# 테스트
accuracy = test_model(model, test_loader)
print(f"Test Accuracy after 18 epochs: {accuracy}")

[0.50739176 0.50739176 0.50739176] [0.25519799 0.25519799 0.25519799]
class number: 7
data number: 28709
batch number: 225
test data number : 7178
test batch number : 57


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/349M [00:00<?, ?B/s]

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This is 1.0th iteration for fine-tuning.

Test Finish!!
Test Accuracy after 15 epochs: 0.3269713011981053
This is 1.0th iteration for fine-tuning.

Test Finish!!
Test Accuracy after 16 epochs: 0.3960713290610198
This is 1.0th iteration for fine-tuning.

Test Finish!!
Test Accuracy after 17 epochs: 0.41529674003900807
This is 1.0th iteration for fine-tuning.

Test Finish!!
Test Accuracy after 18 epochs: 0.43939816104764556
